In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName('TestingRDDS').getOrCreate())

23/02/18 15:38:51 WARN Utils: Your hostname, Dans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.21 instead (on interface en0)
23/02/18 15:38:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 15:38:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
words_list = "Spark make life a lot easier and put me into good Spirits, Spark is too good".split(" ")

In [11]:
type(words_list)

list

In [12]:
print(words_list)

['Spark', 'make', 'life', 'a', 'lot', 'easier', 'and', 'put', 'me', 'into', 'good', 'Spirits,', 'Spark', 'is', 'too', 'good']


In [13]:
words_rdd = spark.sparkContext.parallelize(words_list)

In [14]:
words_data = words_rdd.collect()

In [15]:
for word in words_data:
    print(word)

Spark
make
life
a
lot
easier
and
put
me
into
good
Spirits,
Spark
is
too
good


In [16]:
words_rdd.count()

16

In [17]:
words_rdd.distinct().count()

14

In [18]:
words_unique_rdd = words_rdd.distinct()

In [19]:
for word in words_unique_rdd.collect():
    print(word)

good
life
a
lot
Spark
Spirits,
easier
too
put
and
make
into
is
me


In [21]:
def wordStartsWith(word, letter):
    return word.startswith(letter)

In [22]:
words_rdd.filter(lambda word: wordStartsWith(word, 'S')).collect()

['Spark', 'Spirits,', 'Spark']

In [25]:
num_list = [*range(1,21)]

In [26]:
num_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [27]:
nums_rdd = spark.sparkContext.parallelize(num_list)

In [28]:
nums_squared_rdd = nums_rdd.map(lambda n: (n, n**2))

In [29]:
for element in nums_squared_rdd.collect():
    print(element)

(1, 1)
(2, 4)
(3, 9)
(4, 16)
(5, 25)
(6, 36)
(7, 49)
(8, 64)
(9, 81)
(10, 100)
(11, 121)
(12, 144)
(13, 169)
(14, 196)
(15, 225)
(16, 256)
(17, 289)
(18, 324)
(19, 361)
(20, 400)


In [30]:
words_trd_rdd = words_rdd.map(lambda word: (word, word[0], wordStartsWith(word, 'S')))

In [31]:
for element in words_trd_rdd.collect():
    print(element)

('Spark', 'S', True)
('make', 'm', False)
('life', 'l', False)
('a', 'a', False)
('lot', 'l', False)
('easier', 'e', False)
('and', 'a', False)
('put', 'p', False)
('me', 'm', False)
('into', 'i', False)
('good', 'g', False)
('Spirits,', 'S', True)
('Spark', 'S', True)
('is', 'i', False)
('too', 't', False)
('good', 'g', False)


In [32]:
words_rdd.flatMap(lambda word: list(word)).take(10)

['S', 'p', 'a', 'r', 'k', 'm', 'a', 'k', 'e', 'l']

In [33]:
country_list = [("India", 91), ("USA", 4), ("Greece", 13)]

In [34]:
country_rdd = spark.sparkContext.parallelize(country_list)

In [35]:
srtd_country_list = country_rdd.sortByKey().collect()

In [36]:
for country in srtd_country_list:
    print(country)

('Greece', 13)
('India', 91)
('USA', 4)


In [37]:
srtd_country_list = country_rdd.map(lambda c: (c[1], c[0])).sortByKey(False).collect()

In [38]:
for country in srtd_country_list:
    print(country)

(91, 'India')
(13, 'Greece')
(4, 'USA')


In [39]:
num_list = [1, 5, 2, 4]

In [40]:
result = spark.sparkContext.parallelize(num_list).reduce(lambda x, y: x + y)

In [41]:
print(result)

12


In [42]:
def sum_list(x, y):
    print(x, y)
    return x + y

In [43]:
result = spark.sparkContext.parallelize(num_list).reduce(lambda x, y: sum_list(x,y))
print(result)

1 5
6 2
8 4
12


In [44]:
def wordlengthreducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord

In [45]:
words_rdd.reduce(wordlengthreducer)

'Spirits,'

AttributeError: 'RDD' object has no attribute 'last'

In [50]:
spark.sparkContext.parallelize(range(1, 21)).max()

20

In [51]:
spark.sparkContext.parallelize(range(1, 21)).min()

1

In [52]:
#exercise on daily temparature
# c= (f-32) * 5/9
temp = [("day1", 59), ("day2", 57.2), ("day3", 53.6), ("day4", 55.4), ("day5", 51.8),
       ("day6", 53.6), ("day7", 55.4)]

In [70]:
templist = [59, 57.2, 53.6, 55.4, 51.8, 53.8, 55.4]

In [71]:
tempRDD = spark.sparkContext.parallelize(templist)

In [72]:
def f_to_c(a):
    return (a-32) * 5/9

In [73]:
output = tempRDD.map(f_to_c)


In [74]:
output.collect()

[15.0,
 14.000000000000002,
 12.0,
 13.0,
 10.999999999999998,
 12.111111111111109,
 13.0]

In [75]:
final = output.filter(lambda x: x>= 13)


In [76]:
final.collect()

[15.0, 14.000000000000002, 13.0, 13.0]